In [5]:
#Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt
import os

In [6]:
class VineyardSurrogate:
    def __init__(self, grid_rows=16, grid_cols=12, et_coeff=0.05,
                 optimal_low=0.35, optimal_high=0.75, stages_duration=50,
                 end_on_death=True, diffusion=False, rand_et=True,
                 verbose=True):
        """
        Initialize the vineyard surrogate model

        Parameters:
        - grid_rows, grid_cols: Dimensions of the grid (e.g., 16x12) will be changed to assist with model speed and resolution.
        - et_coeff: Paraemeter to control water loss per step..
        - optimal_low, optimal_high: Bounds for optimal water level, outside of bounds pentalties start
        - stages_duration: Time steps per growth stage.
        - end_on_death: If True, simulation ends early if any crop dies. (should assist with RL training speeds)
        - diffusion: If True, enable water diffusion between cells (not implemented yet).
        - verbose: If True, shows plots of at each stage change
        """

        #User Specicied Variables
        self.rows = grid_rows
        self.cols = grid_cols
        self.optimal_low = optimal_low
        self.optimal_high = optimal_high
        self.stages_duration = stages_duration
        self.total_steps = stages_duration * 4
        self.end_on_death = end_on_death
        self.diffusion = diffusion
        self.et_coeff = np.full((self.rows, self.cols), et_coeff)
        self.rand_et = rand_et
        self.verbose = verbose
        #Modify ET to add some variability
        if self.rand_et:
            self.et_random_factor = np.clip(np.random.normal(1.0, 0.1, (self.rows, self.cols)), 0.8, 1.2)
        else:
            self.et_random_factor = np.ones((self.rows, self.cols))

        #Model Variables
        self.stages = ['Bud Break', 'Flowering', 'Fruit Set', 'Ripening', 'Harvest']
        self.max_water = 1 #Normalized max quantity of water soil can absorb
        self.current_step = 0
        self.current_stage = 0
        self.time_in_stage = np.zeros((self.rows, self.cols), dtype=int)  # Time steps in current stage per cell
        self.alive = np.ones((self.rows, self.cols), dtype=bool)  # Alive/dead per cell
        self.water = np.full((self.rows, self.cols), 0.5)  # Initial even water level
        self.quantity_grid = np.zeros((self.rows, self.cols))  # Per-cell quantity
        self.quality_grid = np.ones((self.rows, self.cols))  # Per-cell quality
        self.total_water_usage = 0.0  # Total irrigation water used (inch-acres)
        self.cum_rain = 0.0

        #Turnable stage parameters (tune to model realistic grape growth across different stages)
        self.stage_params = {
            0: {'qty_rate': 0.00005, 'under_penalty': -0.05, 'over_penalty': 0.1, 'qual_factor': 0.001},
            1: {'qty_rate': 0.0001, 'under_penalty': -0.1, 'over_penalty': 0.5, 'qual_factor': 0.002},
            2: {'qty_rate': 0.0003, 'under_penalty': -0.9, 'over_penalty': 3.5, 'qual_factor': 0.005},
            3: {'qty_rate': 0.00015, 'under_penalty': -0.5, 'over_penalty': 0.5, 'qual_factor': 0.015}
        }

        #Store simulation history for graphing
        self.water_history = []
        self.quantity_history = []
        self.quality_history = []
        self.usage_history = []
        self.rain_history = []  # Per-step rain
        self.cum_rain_history = []  # Cumulative rain over steps

    def reset(self):
        """Reset the simulation to initial state."""
        self.__init__(grid_rows=self.rows, grid_cols=self.cols, et_coeff=self.et_coeff,
                      optimal_low=self.optimal_low, optimal_high=self.optimal_high,
                      stages_duration=self.stages_duration, end_on_death=self.end_on_death,
                      diffusion=self.diffusion, rand_et=self.rand_et)

    def step(self, rainfall=0.0, irrigation=None):
        """
        Advance the simulation by one time step.

        Parameters:
        - rainfall: Uniform rainfall amount (inches) added to all cells.
        - irrigation: 2D array of irrigation amounts per cell (inches).

        Returns:
        - done: True if simulation ended (200 steps or death if enabled).
        - info: Dict with current quantity, quality, water_usage.
        """

        #Check if current step is past total step limit
        if self.current_step >= self.total_steps:
            return True, self.get_info()

        if irrigation is None:
            irrigation = np.zeros((self.rows, self.cols))

        #Add rainfall and irrigation
        self.water += rainfall/6
        self.water += irrigation/6
        self.total_water_usage += np.sum(irrigation) / (self.rows * self.cols)  # Average inch-acres
        self.cum_rain += rainfall

        #Check for crop death due to water
        over_water = self.water > self.max_water #Death by drowning
        under_water = self.water < 0.01  #Death by drought
        dead_cells = over_water | under_water

        #Set cells to dead and set quantity and quality to 0
        self.alive[dead_cells] = False
        #self.quantity_grid[dead_cells] = 0.0
        self.quality_grid[dead_cells] = 0.0

        #Check for dead cells and end on death
        if self.end_on_death and np.any(~self.alive):
            return True, self.get_info()

        #Evapotranspiration loss
        et_loss = self.et_coeff * np.sqrt(self.water) * self.et_random_factor #Simple water loss per step (more water = larger loss)
        self.water = np.maximum(self.water - et_loss, 0.0)

        #Diffusion (spread water to adjacent cells [WIP])
        if self.diffusion:
            pass  #Not implimented yet

        # Update growth
        self.update_growth()

        # Advance time
        self.current_step += 1
        self.time_in_stage += 1

        # Check stage transition
        if self.time_in_stage[0,0] >= self.stages_duration:
            self.current_stage += 1
            self.time_in_stage.fill(0)

            if self.verbose:
              self.print_stage_graphs()

        # Record history
        self.water_history.append(np.mean(self.water))
        self.quantity_history.append(np.sum(self.quantity_grid))
        self.quality_history.append(np.mean(self.quality_grid))
        self.usage_history.append(self.total_water_usage)

        done = self.current_step >= self.total_steps
        return done, self.get_info()

    def update_growth(self):
        """Update quantity and quality based on current water and stage."""

        #get parameters based on growth stage
        params = self.stage_params[self.current_stage]
        alive_mask = self.alive.astype(float)

        #Quantity change
        under = self.water < self.optimal_low
        over = self.water > self.optimal_high

        qty_deltas = (params['qty_rate'] * (1 + params['under_penalty'] * under.astype(float) + params['over_penalty'] * over.astype(float)))
        self.quantity_grid += qty_deltas * alive_mask

        #Quality change

        #Water penalty
        dev_under = np.maximum(self.optimal_low - self.water, 0)
        dev_over = np.maximum(self.water - self.optimal_high, 0)
        water_penalties = params['qual_factor'] * (dev_under + dev_over)

        #Quantity penalty
        total_quantity = np.sum(self.quantity_grid)
        qty_penalty = 0
        if total_quantity > 5:
            qty_penalty = params['qual_factor']* 0.1 * (total_quantity - 5)

        self.quality_grid -= (water_penalties + qty_penalty) * alive_mask
        self.quality_grid = np.maximum(self.quality_grid, 0)

    def get_info(self):
        """Get current simulation info."""
        return {
            'quantity': np.sum(self.quantity_grid),
            'quality': np.mean(self.quality_grid),
            'water_usage': self.total_water_usage,
            'stage': self.stages[self.current_stage],
            'step': self.current_step,
            'any_dead': np.any(~self.alive)}

    def print_stage_graphs(self):
        """Print field-wide graphs every stage change and return image."""

        print(f"Stage {self.stages[self.current_stage - 1]} completed at step {self.current_step}")
        print("Average ET Coefficient:", np.mean(self.et_coeff))
        print("Average Water Level:", np.mean(self.water))
        print("Crop State: Quantity={:.2f}, Quality={:.2f}".format(np.sum(self.quantity_grid), np.mean(self.quality_grid)))

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Water Grid
        im0 = axs[0].imshow(self.water, cmap='Blues', vmin=0, vmax=1)
        axs[0].set_title('Water Grid')
        fig.colorbar(im0, ax=axs[0])

        # Growth Grid: Use -1 for dead cells and set under-color to black
        growth_vis = self.quantity_grid.copy()
        growth_vis[~self.alive] = -1
        cmap = plt.get_cmap('Greens').copy()
        cmap.set_under(color='black')
        im1 = axs[1].imshow(growth_vis, cmap=cmap, vmin=0, vmax=3.0)
        axs[1].set_title('Growth Grid (Black=Dead)')
        fig.colorbar(im1, ax=axs[1])

        # Quality Grid
        im2 = axs[2].imshow(self.quality_grid, cmap='Oranges', vmin=0, vmax=1)
        axs[2].set_title('Quality Grid')
        fig.colorbar(im2, ax=axs[2])

        plt.tight_layout()
        fig.canvas.draw()
        plt.show(fig)
        plt.close('all')

    def plot_histories(self):
        """Plot simulation histories."""
        steps = range(len(self.water_history))
        fig, axs = plt.subplots(6, 1, figsize=(10, 18))

        axs[0].plot(steps, self.water_history)
        axs[0].set_title('Average Water Level Over Time')

        axs[1].plot(steps, self.quantity_history)
        axs[1].set_title('Crop Quantity Over Time')

        axs[2].plot(steps, self.quality_history)
        axs[2].set_title('Crop Quality Over Time')

        axs[3].plot(steps, self.usage_history)
        axs[3].set_title('Total Water Usage Over Time')

        axs[4].step(steps, self.rain_history)
        axs[4].set_title('Rain Amount Per Step')

        axs[5].plot(steps, np.array(self.cum_rain_history) + np.array(self.usage_history))
        axs[5].set_title('Total Added Water (Rain + Irrigation) Over Time')

        plt.tight_layout()
        plt.show()
        plt.close('all')

In [7]:
# RL Agents
class PolicyNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_sizes):
        super().__init__()
        layers = []
        sizes = [input_dim] + hidden_sizes
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
        self.hidden_layers = nn.ModuleList(layers)
        self.output_layer = nn.Linear(hidden_sizes[-1], output_dim)

    def forward(self, obs):
        x = obs
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        logits = self.output_layer(x)
        probs = F.softmax(logits, dim=-1)
        return probs

class CentralizedIrrigationModel:
    def __init__(self, state_dim, lr, gamma):
        self.model = PolicyNet(input_dim=state_dim, output_dim=2, hidden_sizes=[64, 64])  # 0=no irrigate, 1=yes
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma

    def select_action(self, state):
        state = torch.as_tensor(state, dtype=torch.float32)
        probs = self.model(state)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.item(), log_prob

    def compute_returns(self, rewards):
        G = 0.0
        returns = []
        for r in reversed(rewards):
            G = r + self.gamma * G
            returns.append(G)
        returns.reverse()
        return returns

    def update_model(self, log_probs, rewards):
        returns = torch.tensor(self.compute_returns(rewards), dtype=torch.float32)
        loss = sum(-log_p * G for log_p, G in zip(log_probs, returns))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

class LocalIrrigationModel:
    def __init__(self, state_dim, lr, gamma):
        self.model = PolicyNet(input_dim=state_dim, output_dim=4, hidden_sizes=[64, 64])  # 0-3 map to [0.25, 0.5, 0.75, 1.0]
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma

    def select_action(self, state):
        state = torch.as_tensor(state, dtype=torch.float32)
        probs = self.model(state)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.item(), log_prob

    def compute_returns(self, rewards):
        G = 0.0
        returns = []
        for r in reversed(rewards):
            G = r + self.gamma * G
            returns.append(G)
        returns.reverse()
        return returns

    def update_model(self, log_probs, rewards):
        returns = torch.tensor(self.compute_returns(rewards), dtype=torch.float32)
        loss = sum(-log_p * G for log_p, G in zip(log_probs, returns))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

def get_global_state(model, time_since_last_irrigate, forecasted_rain=0.0):
    """Global state for centralized agent."""
    mean_water = np.mean(model.water)
    norm_time_since_irr = time_since_last_irrigate / model.total_steps  # Normalized [0,1]
    return np.array([mean_water, norm_time_since_irr, forecasted_rain])

def get_local_state(model, region_idx, total_irrigation_local, projected_rain=0.0, num_locals=4):
    """Local state for a subgrid."""
    rows_per_local = model.rows // int(np.sqrt(num_locals))
    cols_per_local = model.cols // int(np.sqrt(num_locals))
    row_idx = region_idx // int(np.sqrt(num_locals))
    col_idx = region_idx % int(np.sqrt(num_locals))
    row_start = row_idx * rows_per_local
    row_end = row_start + rows_per_local
    col_start = col_idx * cols_per_local
    col_end = col_start + cols_per_local

    sub_water = model.water[row_start:row_end, col_start:col_end]
    sub_alive = model.alive[row_start:row_end, col_start:col_end]

    mean_water = np.mean(sub_water)
    norm_total_irr = total_irrigation_local / model.total_steps  # Normalized
    mean_health = np.mean(sub_alive)
    return np.array([mean_water, norm_total_irr, projected_rain, mean_health]), (row_start, row_end, col_start, col_end)

def save_models(central, local_agents, path='models/'):
    os.makedirs(path, exist_ok=True)
    torch.save(central.model.state_dict(), f"{path}central.pth")
    for i, agent in enumerate(local_agents):
        torch.save(agent.model.state_dict(), f"{path}local_{i}.pth")

def load_agents(global_state_dim, local_state_dim, lr, gamma, num_locals, path='models/'):
    central = CentralizedIrrigationModel(global_state_dim, lr, gamma)
    central.model.load_state_dict(torch.load(f"{path}central.pth"))

    local_agents = []
    for i in range(num_locals):
        local = LocalIrrigationModel(local_state_dim, lr, gamma)
        local.model.load_state_dict(torch.load(f"{path}local_{i}.pth"))
        local_agents.append(local)

    return central, local_agents

In [8]:
def train_agents(alpha_quantity=1.0, alpha_quality=1.0, alpha_steps=5.0, beta=0.1, num_episodes=1000, lr=0.001, gamma=1, num_locals=4, end_on_death=True, rand_et=True, rain_chance=0.05, rain_amounts=[0.25, 0.5, 0.75, 1.0], rain_probs=[0.45, 0.3, 0.15, 0.1], save_path="./models"):
    env = VineyardSurrogate(end_on_death=end_on_death, rand_et=rand_et)  # Enable early end if desired
    global_state_dim = 3  # mean_water, time_since_last_irr, forecasted_rain
    local_state_dim = 4  # mean_water, total_irr_local, projected_rain, mean_health

    central_agent = CentralizedIrrigationModel(global_state_dim, lr, gamma)
    local_agents = [LocalIrrigationModel(local_state_dim, lr, gamma) for _ in range(num_locals)]

    irrigation_amounts = np.array([0.25, 0.5, 0.75, 1.0])  # For locals when active

    episode_rewards = []  # To track progress
    episode_durations = []  # To track simulation timesteps per episode

    # For plotting
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    plt.ion()  # Interactive mode for live updates in Jupyter

    for episode in range(num_episodes):
        env.reset()
        done = False
        rewards = []
        central_log_probs = []
        central_steps = []  # Timesteps where central acted (all)

        locals_log_probs = [[] for _ in range(num_locals)]
        locals_steps = [[] for _ in range(num_locals)]  # Timesteps where each local acted

        time_since_last_irrigate = 0
        locals_total_irr = [0.0 for _ in range(num_locals)]  # Cumulative per local

        # Tracking histories (per episode)
        central_irrigate_history = []  # 1=yes, 0=no per step
        locals_irrigation_history = [[] for _ in range(num_locals)]  # Cumulative per region per step

        step = 0

        while not done:
            # Sample rain for this step
            if np.random.rand() < rain_chance:
                rainfall = np.random.choice(rain_amounts, p=rain_probs)
            else:
                rainfall = 0.0

            global_state = get_global_state(env, time_since_last_irrigate, forecasted_rain=rainfall)
            irrigate, central_log = central_agent.select_action(global_state)
            central_log_probs.append(central_log)
            central_steps.append(step)
            central_irrigate_history.append(irrigate)

            irrigation_grid = np.zeros((env.rows, env.cols))

            if irrigate == 1:  # Yes: Activate locals
                time_since_last_irrigate = 0  # Reset counter
                for i in range(num_locals):
                    local_state, sub_slice = get_local_state(env, i, locals_total_irr[i], projected_rain=rainfall, num_locals=num_locals)
                    amount_idx, local_log = local_agents[i].select_action(local_state)
                    locals_log_probs[i].append(local_log)
                    locals_steps[i].append(step)

                    amount = irrigation_amounts[amount_idx]
                    r_start, r_end, c_start, c_end = sub_slice
                    irrigation_grid[r_start:r_end, c_start:c_end] = amount

                    locals_total_irr[i] += amount * ((r_end - r_start) * (c_end - c_start)) / (env.rows * env.cols)  # Scaled addition
            else:
                time_since_last_irrigate += 1

            # Update local histories (cumulative even if no action)
            for i in range(num_locals):
                locals_irrigation_history[i].append(locals_total_irr[i])

            done, info = env.step(rainfall=rainfall, irrigation=irrigation_grid)
            #Reward for keeping crops alive
            alive_fraction = np.mean(env.alive)
            reward = - (1 - alive_fraction)  # Dense: -1 if all dead, 0 if all alive
            if done:
                reward += alpha_quantity * info['quantity'] + alpha_quality * info['quality'] - beta * info['water_usage']  + alpha_steps * env.current_step
            rewards.append(reward)

            step += 1

        episode_rewards.append(sum(rewards))
        episode_durations.append(env.current_step)

        # Update central (full episode)
        central_agent.update_model(central_log_probs, rewards)

        # Update locals (only if they acted; compute returns from their action steps onward)
        for i in range(num_locals):
            if locals_log_probs[i]:
                local_returns = []
                for act_step in locals_steps[i]:
                    G = 0.0
                    for r in rewards[act_step:]:
                        G = r + gamma * G
                    local_returns.append(G)
                local_agents[i].update_model(locals_log_probs[i], local_returns)

        if episode % 1 == 0:
            print(f"Episode {episode}: Reward = {episode_rewards[-1]}, Quantity={info['quantity']}, Quality={info['quality']}, Usage={info['water_usage']}, Early End={env.current_step < env.total_steps}")

        # Live plot updates every episode
        episodes = np.arange(episode + 1)

        # Duration plot
        axs[0].cla()
        axs[0].plot(episodes, episode_durations, label='Raw Duration', color='blue')
        if episode >= 9:
            moving_avg_dur = np.convolve(episode_durations, np.ones(10)/10, mode='valid')
            axs[0].plot(episodes[9:], moving_avg_dur, label='10-Ep Avg', color='red')
        axs[0].set_title('Simulation Duration vs Episode')
        axs[0].set_xlabel('Episode')
        axs[0].set_ylabel('Timesteps')
        axs[0].legend()

        # Reward plot
        axs[1].cla()
        axs[1].plot(episodes, episode_rewards, label='Raw Reward', color='green')
        if episode >= 9:
            moving_avg_rew = np.convolve(episode_rewards, np.ones(10)/10, mode='valid')
            axs[1].plot(episodes[9:], moving_avg_rew, label='10-Ep Avg', color='orange')
        axs[1].set_title('Reward vs Episode')
        axs[1].set_xlabel('Episode')
        axs[1].set_ylabel('Reward')
        axs[1].legend()

        fig.canvas.draw()

    # Final plots (same as live, but static)
    plt.ioff()

    # Plot decision tracking for last episode (pad to 200 steps if early end)
    max_steps = env.total_steps
    steps = np.arange(max_steps)

    # Pad histories
    padded_central = np.pad(central_irrigate_history, (0, max_steps - len(central_irrigate_history)), mode='constant', constant_values=central_irrigate_history[-1] if central_irrigate_history else 0)
    padded_locals = []
    for hist in locals_irrigation_history:
        padded = np.pad(hist, (0, max_steps - len(hist)), mode='constant', constant_values=hist[-1] if hist else 0)
        padded_locals.append(padded)

    fig_dec, axs_dec = plt.subplots(1, 2, figsize=(12, 5))

    axs_dec[0].step(steps, padded_central, where='post')
    axs_dec[0].set_title('Central Agent: Irrigate Decisions Over Time')
    axs_dec[0].set_xlabel('Step')
    axs_dec[0].set_ylabel('Irrigate (1=Yes, 0=No)')
    axs_dec[0].set_xlim(0, max_steps)

    for i in range(num_locals):
        axs_dec[1].plot(steps, padded_locals[i], label=f'Region {i}')
    axs_dec[1].set_title('Local Agents: Cumulative Irrigation Over Time')
    axs_dec[1].set_xlabel('Step')
    axs_dec[1].set_ylabel('Cumulative Irrigation')
    axs_dec[1].legend()
    axs_dec[1].set_xlim(0, max_steps)

    plt.tight_layout()
    plt.show()

    if save_path:
        save_models(central_agent, local_agents, save_path)

    return central_agent, local_agents, episode_rewards, episode_durations

In [1]:
central, locals_list, rewards_history, episode_durations = train_agents(alpha_quantity=1.0, alpha_quality=1.0, alpha_steps=5.0, beta=0.1, num_episodes=1000, lr=0.001, gamma=1, num_locals=4, end_on_death=True, rand_et=True, rain_chance=0.05, rain_amounts=[0.25, 0.5, 0.75, 1.0], rain_probs=[0.45, 0.3, 0.15, 0.1], save_path="./models")

NameError: name 'train_agents' is not defined